In [3]:
# cargando bibliotecas requeridas para realizar análisis de sentimiento

import pyarrow as pa
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from textblob import TextBlob

In [4]:
# leyendo archivo

user_reviews = pq.read_table('user_reviews.parquet')

In [5]:
# Mostrar las primeras filas de cada DataFrame
print("Primeras y ultimas filas de user_reviews:")
user_reviews

Primeras y ultimas filas de user_reviews:


pyarrow.Table
user_id: string
item_id: int64
recommend: bool
review: string
__index_level_0__: int64
----
user_id: [["76561197970982479","76561197970982479","76561197970982479","js41637","js41637",...,"76561198312638244","76561198312638244","LydiaMorley","LydiaMorley","LydiaMorley"]]
item_id: [[1250,22200,43110,251610,227300,...,70,362890,273110,730,440]]
recommend: [[true,true,true,true,true,...,true,true,true,true,true]]
review: [["Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.","It's unique and worth a playthrough.","Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!","I know what you think when you see this title "Barbie Dreamhouse Party" but do not be 

In [6]:
# crear una función a la que llamar para analizar cada reseña y clasificarla dentro de los parámetros dados

def sentiment_analysis(text):
    
    analysis = TextBlob(str(text))
    score = analysis.sentiment.polarity

    if score > 0:
        return 2  # Positive
    elif score < 0:
        return 0  # Negative
    else:
        return 1  # Neutral


In [7]:
# Convirtiendo user_reviews en un dataframe de pandas

user_reviews_pandas = user_reviews.to_pandas()

In [8]:
# Aplicar la función de análisis de sentimiento a la columna "review" y crear la columna "sentiment_analysis"

user_reviews_pandas['sentiment_analysis'] = np.where(user_reviews_pandas['review'].notna(), user_reviews_pandas['review'].apply(lambda x: sentiment_analysis(x)), 1)

In [9]:
# Mostrar las primeras filas de cada DataFrame
print("Primeras y ultimas filas de user_reviews:")
user_reviews_pandas

Primeras y ultimas filas de user_reviews:


,user_id,item_id,recommend,review,sentiment_analysis
0,76561197970982479,1250,True,Simple yet with great replayability. In my opi...,2
1,76561197970982479,22200,True,It's unique and worth a playthrough.,2
2,76561197970982479,43110,True,Great atmosphere. The gunplay can be a bit chu...,2
3,js41637,251610,True,I know what you think when you see this title ...,2
4,js41637,227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...
59300,76561198312638244,70,True,a must have classic from steam definitely wort...,2
59301,76561198312638244,362890,True,this game is a perfect remake of the original ...,2
59302,LydiaMorley,273110,True,had so much fun plaing this and collecting res...,2
59303,LydiaMorley,730,True,:D,2


In [10]:
# Verificar si hay algún valor no nulo en la columna 'review'
hay_valores_no_nulos = user_reviews_pandas['review'].isnull().any()

if hay_valores_no_nulos:
    print("Hay al menos un valor nulo en la columna 'review'.")
else:
    print("Ninguno de los valores en la columna 'review' son NaN.")


Ninguno de los valores en la columna 'review' son NaN.


In [11]:
# eliminar la columna "review"

user_reviews_pandas = user_reviews_pandas.drop('review', axis=1)

In [12]:
user_reviews_pyarrow = pa.Table.from_pandas(user_reviews_pandas)
pq.write_table(user_reviews_pyarrow, "user_reviews.parquet")


